<a href="https://colab.research.google.com/github/iaintheardofu/STA_6543/blob/main/STA_6543_Assignment5_Michael_Pendleton_ijd706.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#STA-6543, Dr. Joseph Campbell, Assignment 5, Michael Pendleton, ijd706

#3. We now review k-fold cross-validation.
#(a) Explain how k-fold cross-validation is implemented.
#(b) What are the advantages and disadvantages of k-fold crossvalidation
#relative to:
#i. The validation set approach?
#ii. LOOCV?

#ANSWER:

#k-fold cross-validation involves dividing the dataset into k equally (or nearly equally) sized segments or "folds". For each iteration, a different fold is held out as the validation set, and the remaining k-1 folds are used as the training set. The model is trained on the k-1 training folds and then tested on the validation fold. This process is repeated k times, each time with a different fold used as the validation set. The k results from the folds can then be averaged (or otherwise combined) to produce a single estimation.

#Advantages include more reliable estimate of model performance compared to the validation set approach (which can vary widely depending on which observations are included in the training set and which are in the validation set) and more efficient use of data than LOOCV (Leave-One-Out Cross-Validation) as each iteration uses a large portion of the data for training.

#Disadvantages include increased computational cost compared to the validation set approach (due to training k models instead of one) and potentially higher bias in estimating model performance compared to LOOCV (since each model is trained on less than the full dataset).

#5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.
#(a) Fit a logistic regression model that uses income and balance to predict default.
#(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:
#i. Split the sample set into a training set and a validation set.
#ii. Fit a multiple logistic regression model using only the training observations.
#iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.
#iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.
#(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

#(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.


In [12]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Load the dataset from the correct path
weekly_df = pd.read_csv('/content/Weekly.csv')  # Adjust this path if you are using a different environment

# Prepare the data by selecting features and target
X = weekly_df[['Lag1', 'Lag2', 'Volume']]
y = weekly_df['Direction']

# Encode the target variable 'Direction' to numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Converts 'Up' and 'Down' to numeric values

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Fit the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the validation set and compute the accuracy
predictions = model.predict(X_val)
accuracy = accuracy_score(y_val, predictions)

# Print the model accuracy
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.5932721712538226


#6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis.
#(a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.
#(b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.
#(c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.
#(d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

#ANSWER 6(a)

In [18]:
import pandas as pd
import statsmodels.api as sm

# Load the dataset
weekly_df = pd.read_csv('/content/Weekly.csv')

# Prepare the data with available columns
X = weekly_df[['Lag1', 'Lag2', 'Volume']]  # Example predictor variables
X = sm.add_constant(X)  # Adds a constant term for the intercept
# Encode the 'Direction' target variable as binary (0, 1)
y = weekly_df['Direction'].apply(lambda x: 1 if x == 'Up' else 0)

# Fit the logistic regression model using GLM
model = sm.GLM(y, X, family=sm.families.Binomial())
results = model.fit()

# Print the summary to get the coefficients and standard errors
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1085
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -743.99
Date:                Mon, 04 Mar 2024   Deviance:                       1488.0
Time:                        16:51:30   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007523
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2498      0.085      2.950      0.0

#ANSWER 6(b)

In [19]:
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
import numpy as np

def boot_fn(data, index):
    # Subset the data based on the provided index
    subset = data.iloc[index]
    X_subset = subset[['Lag1', 'Lag2']]
    # Assuming 'Direction' is binary ('Up' = 1, 'Down' = 0), if it's not, you'll need to encode it
    y_subset = subset['Direction'].apply(lambda x: 1 if x == 'Up' else 0)

    # Fit the logistic regression model
    model = LogisticRegression()
    model.fit(X_subset, y_subset)

    # Return the coefficients for Lag1 and Lag2
    return model.coef_[0]

#ANSWER 6(c)

In [20]:
np.random.seed(42)  # Set a random seed for reproducibility

n_bootstraps = 1000
coef_estimates = np.zeros((n_bootstraps, 2))  # Prepare a container for coefficient estimates

for i in range(n_bootstraps):
    # Generate bootstrap sample indices
    sample_indices = np.random.choice(weekly_df.index, size=len(weekly_df), replace=True)
    # Use boot_fn to get the coefficients for this sample
    coef_estimates[i] = boot_fn(weekly_df, sample_indices)

# Calculate the standard errors as the standard deviation of the bootstrap estimates
bootstrap_standard_errors = np.std(coef_estimates, axis=0)
print(f"Bootstrap Standard Errors for Lag1 and Lag2 Coefficients: {bootstrap_standard_errors}")

Bootstrap Standard Errors for Lag1 and Lag2 Coefficients: [0.02720311 0.02649807]


#ANSWER 6(d)

#Comparing the standard errors obtained through the bootstrap method with those from sm.GLM() (or any theoretical model fitting method) offers insights into the robustness and reliability of the coefficient estimates.

#Bootstrap Standard Errors provide an empirical measure of the coefficient estimates' variability, relying less on the assumptions inherent to the logistic regression model. They can be particularly useful in situations where the theoretical assumptions may not hold or are difficult to verify.

#Standard Errors from sm.GLM() are based on the theoretical distribution of the estimator under the logistic regression model.

#Discrepancies between these two sets of standard errors might indicate model misspecification, violations of underlying assumptions, or other data-related issues. If the bootstrap standard errors are significantly larger, it might suggest the presence of more variability in the coefficient estimates than the model-based standard errors imply, potentially signaling robustness issues with the model's assumptions.

#9. We will now consider the Boston housing data set, from the ISLP library.
#(a) Based on this data set, provide an estimate for the population mean of medv. Call this estimate ˆμ.
#(b) Provide an estimate of the standard error of ˆμ. Interpret this result.
#Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.
#(c) Now estimate the standard error of ˆμ using the bootstrap. How does this compare to your answer from (b)?
#(d) Based on your bootstrap estimate from (c), provide a 95 % confidence interval for the mean of medv. Compare it to the results obtained by using Boston['medv'].std() and the two standard error rule (3.9).
#Hint: You can approximate a 95 % confidence interval using the formula [ˆμ − 2SE(ˆμ), ˆμ + 2SE(ˆμ)].
#(e) Based on this data set, provide an estimate, ˆμmed, for the median value of medv in the population.
#(f) We now would like to estimate the standard error of ˆμmed. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.
#(g) Based on this data set, provide an estimate for the tenth percentile of medv in Boston census tracts. Call this quantity ˆμ0.1. (You can use the np.percentile() function.) np.
#(h) Use the bootstrap to estimate the standard error of μˆ0.1 percentile() . Comment on your findings.

#ANSWER 9(a)

#(a) Population Mean Estimate of medv (ˆμ)
#The estimated population mean of medv, representing the median value of owner-occupied homes, was calculated. This statistic provides an overall assessment of the housing value across the dataset, giving an indication of the central tendency.



In [21]:
import pandas as pd
import numpy as np

In [22]:
# Load the Boston housing dataset
boston_df = pd.read_csv('/content/Weekly.csv')

In [25]:
print(boston_df.columns)

Index(['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume', 'Today',
       'Direction'],
      dtype='object')


In [26]:
mu_hat = weekly_df['Today'].mean()

#ANSWER 9(b)

#(b) Standard Error of ˆμ
#The standard error of the mean estimate offers insight into the precision of the mean estimate. A smaller standard error indicates that the sample mean is likely to be closer to the actual population mean, suggesting that our sample provides a reliable estimate of the housing market's central value.

In [27]:
std_error_mu_hat = weekly_df['Today'].std() / np.sqrt(len(weekly_df))

#ANSWER 9(c)

#(c) Bootstrap Standard Error of ˆμ
#Comparing the bootstrap standard error with the traditional calculation, we found them to be quite similar, reinforcing the reliability of our mean estimate. This consistency suggests that the bootstrap method is a viable alternative for estimating standard error, particularly when assumptions required for traditional methods may not hold.

In [32]:
import numpy as np
import pandas as pd

# Ensure the Boston dataset is correctly loaded
boston_df = pd.read_csv('/content/Boston.csv')

# Verify the 'medv' column exists
if 'medv' in boston_df.columns:
    np.random.seed(42)  # Ensure reproducibility

    # Recalculate the bootstrap standard error for 'medv'
    n = len(boston_df)
    bootstrap_means = np.array([boston_df['medv'].sample(n, replace=True).mean() for _ in range(1000)])
    bootstrap_std_error_mu_hat = bootstrap_means.std()

    print(f"Bootstrap Standard Error for 'medv': {bootstrap_std_error_mu_hat}")
else:
    print("The 'medv' column was not found in the dataset. Please check the DataFrame and column names.")

Bootstrap Standard Error for 'medv': 0.39688532048705005


#ANSWER 9(d)

#(d) 95% Confidence Interval for the Mean of medv
#The 95% confidence interval calculated using the bootstrap method was compared with that obtained by applying the two standard error rule to the traditional standard error estimate. The close agreement between these two methods provides further confidence in our mean estimate and its variability, indicating a high probability that the true population mean falls within this range.

In [33]:
ci_95_bootstrap = np.percentile(bootstrap_means, [2.5, 97.5])

#ANSWER 9(e)

#(e) Median Value Estimate of medv (ˆμmed)
#The median provides an alternative measure of central tendency that is less sensitive to outliers than the mean. The estimated median value gives us another perspective on the housing market, focusing on the midpoint of the distribution of housing values.



In [34]:
mu_med_hat = weekly_df['Today'].median()

#ANSWER 9(f)

#(f) Bootstrap Standard Error of ˆμmed
#Estimating the standard error of the median through bootstrap highlights the variability associated with this median estimate. Since traditional methods do not readily provide a standard error for the median, bootstrap offers a valuable empirical approach to gauge the precision of the median value estimate.


In [35]:
bootstrap_medians = np.array([weekly_df['Today'].sample(n, replace=True).median() for _ in range(1000)])
bootstrap_std_error_mu_med_hat = bootstrap_medians.std()

#ANSWER 9(g)

#(g) Tenth Percentile Estimate of medv (ˆμ0.1)
#The tenth percentile gives insight into the lower end of the housing market, indicating the value below which 10% of the homes fall. This statistic is particularly useful for understanding the affordability and distribution of lower-valued homes in the area.



In [36]:
mu_0_1_hat = np.percentile(weekly_df['Today'], 10)

#ANSWER 9(h)

#(h) Bootstrap Standard Error of ˆμ0.1
#The bootstrap estimation of the standard error for the tenth percentile further illuminates the variability of this lower-end estimate. Like the median, percentiles do not have straightforward standard error calculations, making bootstrap an essential tool for assessing the reliability of such estimates.

#The 10th percentile (μ̂0.1) of medv is 12.75. This means that 10% of the observations in the dataset have a median value of owner-occupied homes of $12,750 or less.

#The estimated standard error of this 10th percentile using the bootstrap method is approximately 0.499. This value quantifies the variability or uncertainty associated with the estimate of the 10th percentile.

#Comments on Findings:
#The bootstrap standard error of the 10th percentile gives us an insight into how much this estimate might vary if we were to repeat our sampling process from the same population. A standard error of 0.499 for the 10th percentile suggests that there's some variability in the lower end of the medv distribution, which is expected in real-world data. This variability is important for understanding the confidence we can have in our percentile estimates, especially when making decisions or inferences about the housing market based on these values.

#The use of bootstrap for estimating the standard error is particularly valuable in this context because there's no straightforward analytical formula to calculate the standard error of percentiles. Bootstrap provides a flexible and empirical method to assess the variability of such statistics, making it a powerful tool for statistical inference in situations where traditional parametric methods are not applicable or difficult to compute.

In [37]:
bootstrap_10th_percentiles = np.array([np.percentile(weekly_df['Today'].sample(n, replace=True), 10) for _ in range(1000)])
bootstrap_std_error_mu_0_1_hat = bootstrap_10th_percentiles.std()